# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq
```

In [60]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq
!pip install pyarabic

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Dataset.csv             
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.8 MB/s eta 0:00:00


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [61]:
import os
import pandas as pd
import string
import pyarabic.araby as araby
import langchain_huggingface
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [62]:
df = pd.read_csv("Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [63]:
dir = "/content/data/"
os.makedirs(dir, exist_ok=True)

def preprocessing(text):
  translator = str.maketrans('', '', string.punctuation)
  text=text.translate(translator)
  text=araby.strip_tashkeel(text)
  text=araby.strip_harakat(text)
  text=araby.strip_tatweel(text)
  text=" ".join(text.split())
  return text

markdown_files = []
for i in range(0, len(df)):
  markdown_content = f"**{preprocessing(df['المخالفة'].iloc[i])}** - {df['الغرامة'].iloc[i]}"
  markdown_files.append(markdown_content)

In [64]:
markdown_files[0:10]

['**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها** - الغرامة المالية 100 - 150 ريال',
 '**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها** - الغرامة المالية 100 - 150 ريال',
 '**عدم وجود تأمين ساري للمركبة** - الغرامة المالية 100 - 150 ريال',
 '**عبور المشاة للطرق من غير الأماكن المخصصة لهم** - الغرامة المالية 100 - 150 ريال',
 '**عدم تقيد المشاة بالإشارات الخاصة بهم** - الغرامة المالية 100 - 150 ريال',
 '**وقوف المركبة في أماكن غير مخصصة للوقوف** - الغرامة المالية 100 - 150 ريال',
 '**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم** - الغرامة المالية 100 - 150 ريال',
 '**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار** - الغرامة المالية 150 - 300 ريال',
 '**الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين مترا** - الغرامة المالية 150 - 300 ريال',
 '**قيام سائق الدراجة الآلية أو العادية أو ما في حكمهما بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر** - الغرامة المالية 150 - 300 ريال']

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [65]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

docs = text_splitter.create_documents(markdown_files)

In [66]:
docs[0:10]

[Document(metadata={}, page_content='**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم وجود تأمين ساري للمركبة** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عبور المشاة للطرق من غير الأماكن المخصصة لهم** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم تقيد المشاة بالإشارات الخاصة بهم** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**وقوف المركبة في أماكن غير مخصصة للوقوف** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم** - الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار** - الغرامة المالية 150

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [67]:
embeddings = HuggingFaceEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")

database = Chroma.from_documents(
    documents = docs,
    embedding = embeddings,
    persist_directory='/content/data/chroma.db'
)

## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [68]:
ar_prompt_template= """
* بالبداية رحب بالمستخدم قبل كل شيء
* اخبر المستخدم ان البيانات المعطاه قد تكون غير صحيحة
* انصح المستخدم بزيارة موقع المرور الرئيسي للتأكد
* بالأضافة, قدم نصيحه للمستخدم باللغة العربية عن السؤال المعطى {context}
----
context: {context}
question: {question}
"""

prompt = PromptTemplate(
    template= ar_prompt_template,
    input_variables=['context', 'question']

)


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [69]:
api_key = ""

In [70]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    api_key=api_key
)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [71]:
model = LLMChain(
    llm= llm,
    prompt=prompt,
    verbose=True
)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [92]:
def query_rag(query: str):
  similarity_result = database.similarity_search(query, k=1)
  rag_res = model.invoke(
                          {
                           "context": similarity_result[0].page_content,
                           "question": query
                           })

  return rag_res['text']

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [93]:
print(query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة"))

Prompt after formatting:

* بالبداية رحب بالمستخدم قبل كل شيء
* اخبر المستخدم ان البيانات المعطاه قد تكون غير صحيحة
* انصح المستخدم بزيارة موقع المرور الرئيسي للتأكد
* بالأضافة, قدم نصيحه للمستخدم باللغة العربية عن السؤال المعطى **عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.** - الغرامة المالية 500 - 900 ريال
----
context: **عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.** - الغرامة المالية 500 - 900 ريال
question: ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة


> Finished chain.
مرحباً بك!

أود أن أذكرك أن البيانات المعطاة قد تكون غير صحيحة أو قد تكون قد تغيرت منذ آخر تحديث، لذلك أوصي بزيارة موقع المرور الرئيسي للتأكد من دقة المعلومات.

بالنسبة إلى سؤالك حول عقوبة عدم الوقوف وقوفاً تاماً عند إشارة "أمامك أفضلية" في حالة مرور مركبات على الطريق المعطاة له الأفضلية، فإن الغرامة المالية قد تتراوح بين 500 إلى 900 ريال.

نصيحتي لك هي أن تأخذ دائماً احتياطات السلامة على الطريق، وتلتزم

In [94]:
print(query_rag("ماهي الغرامة على القيادة بدون رخصة؟"))

Prompt after formatting:

* بالبداية رحب بالمستخدم قبل كل شيء
* اخبر المستخدم ان البيانات المعطاه قد تكون غير صحيحة
* انصح المستخدم بزيارة موقع المرور الرئيسي للتأكد
* بالأضافة, قدم نصيحه للمستخدم باللغة العربية عن السؤال المعطى **قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.** - الغرامة المالية 1000 - 2000 ريال
----
context: **قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.** - الغرامة المالية 1000 - 2000 ريال
question: ماهي الغرامة على القيادة بدون رخصة؟


> Finished chain.
مرحباً بك!

أود أن أذكرك أن البيانات المعطاة قد تكون غير صحيحة أو غير محدثة، لذلك أوصيك بزيارة موقع المرور الرئيسي للتأكد من المعلومات.

بالنسبة إلى سؤالك عن الغرامة على القيادة بدون رخصة، لم يتم ذكرها في السياق المحدد. ومع ذلك، يمكنني أن أقدم لك نصيحة حول السؤال المعطى: **قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**

تذكر دائمًا أن القيادة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها يمكن أن تؤدي إلى غرامات مالية كبيرة، تتراوح بين 1000 إلى 200